In [1]:
from utils import Logger,adjust_learning_rate,CreateLogger,create_cosine_learning_schedule,encode_test_label,set_seed
from model import Resnet_with_uncertainty_clic#不确定度
from dependency import *
from torch import optim
from torchcontrib.optim import SWA
from dataloader import generate_dataloader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error #

In [2]:
def train(net, train_dataloader, model_name):
    net.set_mode("train")
    train_loss = 0
    train_dia_acc = 0
    train_sps_acc = 0
    for index,(clinic_image,derm_image,meta_data,label) in enumerate(train_dataloader):
        opt.zero_grad()
        
        clinic_image = clinic_image.cuda()
        derm_image = derm_image.cuda()
        #meta_data = meta_data.cuda()
        
        #Diagnostic label
        diagnosis_label = label[0].long().cuda()
        #Seven-Point Checklist labels
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        #uncertainty
        
        [(logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic)] = net((clinic_image))
        
        #clic
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_clic),dim = 1)
        
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_clic),dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_clic),dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_clic), dim = 1)

        prediction5 =torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_clic), dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_clic), dim = 1)

        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_clic), dim = 1)

        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)

        diagnosis_label_t = diagnosis_label.unsqueeze(0)
        pn_label_t = pn_label.unsqueeze(0)
        str_label_t = str_label.unsqueeze(0)
        pig_label_t = pig_label.unsqueeze(0)
        rs_label_t = rs_label.unsqueeze(0)
        dag_label_t = dag_label.unsqueeze(0)
        bwv_label_t = bwv_label.unsqueeze(0)
        vs_label_t = vs_label.unsqueeze(0)
            
        #
        pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
        str_label_t = np.array(str_label_t.cpu(),dtype="int")
        pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
        rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
        dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
        bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
        vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
        diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")

        t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
        t = np.hstack((t,str_label_t.T))
        t = np.hstack((t,pig_label_t.T))
        t = np.hstack((t,rs_label_t.T))
        t = np.hstack((t,dag_label_t.T))
        t = np.hstack((t,bwv_label_t.T))
        t = np.hstack((t,vs_label_t.T))
        t = torch.tensor(t)
            
        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        
        certainty_b = (t == p).long()
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        uncertainty_label_clic = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >=0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label_clic.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label_clic = torch.tensor(uncertainty_label_clic)
        #uncertainty
        uncertainty_label_clic = uncertainty_label_clic.float().cuda()
        
        loss_clic = torch.div(
          net.criterion(logit_diagnosis_clic, diagnosis_label)
        + net.criterion(logit_pn_clic, pn_label)
        + net.criterion(logit_str_clic, str_label)
        + net.criterion(logit_pig_clic, pig_label)
        + net.criterion(logit_rs_clic, rs_label)
        + net.criterion(logit_dag_clic, dag_label)
        + net.criterion(logit_bwv_clic, bwv_label)
        + net.criterion(logit_vs_clic, vs_label)
        + net.criterion_MSE(logit_uncertainty_clic,uncertainty_label_clic), 9)
        
        loss = loss_clic 
        
        dia_acc_all = torch.div(net.metric(logit_diagnosis_clic,diagnosis_label),int(clinic_image.size(0)))
        sps_acc_all = torch.div(  net.metric(logit_pn_clic,pn_label)
                                + net.metric(logit_str_clic,str_label)
                                + net.metric(logit_pig_clic,pig_label)
                                + net.metric(logit_rs_clic,rs_label)
                                + net.metric(logit_dag_clic,dag_label)
                                + net.metric(logit_bwv_clic,bwv_label)
                                + net.metric(logit_vs_clic,vs_label),int(7 * clinic_image.size(0)))
        loss.backward()
        opt.step()

        train_loss += loss.item()
        train_dia_acc += dia_acc_all.item()
        train_sps_acc += sps_acc_all.item()

    train_loss = train_loss / (index + 1) # Because the index start with the value 0f zero
    train_dia_acc = train_dia_acc / (index + 1)
    train_sps_acc = train_sps_acc / (index + 1)

    return train_loss,train_dia_acc,train_sps_acc

In [3]:
def validation(net,val_dataloader,model_name):
    net.set_mode('valid')
    val_loss = 0
    val_dia_acc = 0
    vaL_sps_acc = 0
    for index, (clinic_image, derm_image, meta_data, label) in enumerate(val_dataloader):

        clinic_image = clinic_image.cuda()
        derm_image   = derm_image.cuda()
        #meta_data    = meta_data.cuda()

        diagnosis_label = label[0].long().cuda()
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        

        with torch.no_grad():

            [(logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic)] = net((clinic_image))
        
            diagnosis_label_t = diagnosis_label.unsqueeze(0)
            pn_label_t = pn_label.unsqueeze(0)
            str_label_t = str_label.unsqueeze(0)
            pig_label_t = pig_label.unsqueeze(0)
            rs_label_t = rs_label.unsqueeze(0)
            dag_label_t = dag_label.unsqueeze(0)
            bwv_label_t = bwv_label.unsqueeze(0)
            vs_label_t = vs_label.unsqueeze(0)
        
            #
            pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
            str_label_t = np.array(str_label_t.cpu(),dtype="int")
            pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
            rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
            dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
            bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
            vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
            diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
        
            t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
            t = np.hstack((t,str_label_t.T))
            t = np.hstack((t,pig_label_t.T))
            t = np.hstack((t,rs_label_t.T))
            t = np.hstack((t,dag_label_t.T))
            t = np.hstack((t,bwv_label_t.T))
            t = np.hstack((t,vs_label_t.T))
            t = torch.tensor(t)
         
            #clinic
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis_clic), dim = 1)
        
            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn_clic), dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str_clic), dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig_clic), dim = 1)

            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs_clic), dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag_clic), dim = 1)
    
            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv_clic), dim = 1)

            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs_clic), dim = 1)

            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")
        
            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)
        
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label_clic = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >=0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label_clic.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label_clic = torch.tensor(uncertainty_label_clic)
            #uncertainty
            uncertainty_label_clic = uncertainty_label_clic.float().cuda()
        
            loss_clic = torch.div(
              net.criterion(logit_diagnosis_clic, diagnosis_label)
            + net.criterion(logit_pn_clic, pn_label)
            + net.criterion(logit_str_clic, str_label)
            + net.criterion(logit_pig_clic, pig_label)
            + net.criterion(logit_rs_clic, rs_label)
            + net.criterion(logit_dag_clic, dag_label)
            + net.criterion(logit_bwv_clic, bwv_label)
            + net.criterion(logit_vs_clic, vs_label)
            + net.criterion_MSE(logit_uncertainty_clic,uncertainty_label_clic), 9)

            loss = loss_clic
            
            dia_acc_all = torch.div(net.metric(logit_diagnosis_clic,diagnosis_label),int(clinic_image.size(0)))
            sps_acc_all = torch.div(net.metric(logit_pn_clic,pn_label)
                                + net.metric(logit_str_clic,str_label)
                                + net.metric(logit_pig_clic,pig_label)
                                + net.metric(logit_rs_clic,rs_label)
                                + net.metric(logit_dag_clic,dag_label)
                                + net.metric(logit_bwv_clic,bwv_label)
                                + net.metric(logit_vs_clic,vs_label),int(7 * clinic_image.size(0)))
            
                                    
        val_loss += loss.item()
        val_dia_acc += dia_acc_all.item()
        vaL_sps_acc += sps_acc_all.item()

    val_loss = val_loss / (index + 1)
    val_dia_acc = val_dia_acc / (index + 1)
    vaL_sps_acc = vaL_sps_acc / (index + 1)

    return val_loss,val_dia_acc,vaL_sps_acc

In [4]:
def run_train(model_name,mode,i):
    log.write('** start Resnet_with_Uncertainty(clinic) training here! **\n')
    #best_acc = 0
    es = 0
    patience = 50
    best_mean_acc = 0 
    best_loss = 300
    
    for epoch in range(epochs):
        swa_lr = cosine_learning_schedule[epoch]
        adjust_learning_rate(opt, swa_lr)

        # train_mode
        train_loss,train_dia_acc,train_sps_acc=train(net, train_dataloader,model_name)
        log.write('Round: {}, epoch: {}, Train Loss: {:.4f}, Train Dia Acc: {:.4f}, Train SPS Acc: {:.4f}\n'.format(i, epoch, train_loss,
                                                                                                         train_dia_acc,
                                                                                                         train_sps_acc))

        # validation mode
        val_loss,val_dia_acc,val_sps_acc = validation(net, val_dataloader,model_name)
        
        val_acc = (val_dia_acc + val_sps_acc) / 2
        val_mean_acc = (val_dia_acc*1 + val_sps_acc*7)/8
        
        log.write('Round: {}, epoch: {}, Valid Loss: {:.4f}, Valid Dia Acc: {:.4f}, Valid SPS Acc: {:.4f}\n'.format(i, epoch, val_loss,
                                                                                                         val_dia_acc,
                                                                                                         val_sps_acc))

     
        if val_mean_acc > best_mean_acc:
            es = 0
            best_mean_acc = val_mean_acc
            torch.save(net.state_dict(), out_dir + '/checkpoint/{}_model.pth'.format('best_mean_acc'))
            log.write('Current Best Mean Acc is {}'.format(best_mean_acc))
        #  else:
        #      es += 1
        #      print("Counter {} of {}".format(es,patience))
          
        #      if es > patience:
        #          print("Early stopping with best_mean_acc: {:.4f}".format(best_mean_acc), "and val_mean_acc for this epoch: {:.4f}".format(val_mean_acc))
        #          break
  
  
        if epoch > (epochs - swa_epoch) and epoch % 1 == 0:
            opt.update_swa()
            log.write('SWA Epoch: {}'.format(epoch))

    torch.save(net.state_dict(), out_dir+'/swa_{}_resnet50_model.pth'.format(mode))

In [5]:
if __name__ == '__main__':
    # Hyperparameters
    mode = 'multimodal'
    tech = "clinic"
    model_name = str('Resnet')+str("_")+str(tech)
    shape = (229, 229)
    batch_size = 5
    num_workers = 0
    data_mode = 'Normal'
    deterministic = True
    if deterministic:
        if data_mode == 'Normal':
            random_seeds = 170
        elif data_mode == 'self_evaluated':
            random_seeds = 183
    rounds = 1
    lr = 4.9e-5##
    epochs = 250
    swa_epoch = 150

    train_dataloader, val_dataloader = generate_dataloader(shape, batch_size, num_workers, data_mode,"concat")
    
    for i in range(rounds):
        if deterministic:
            set_seed(random_seeds + i)
      # create logger
        print(random_seeds+i)
        log, out_dir = CreateLogger(mode, model_name,i,data_mode)
        #network 
        net = Resnet_with_uncertainty_clic(class_list).cuda()
        
      # create optimizer
        optimizer = optim.Adam(net.parameters(), lr=lr,weight_decay=5e-5)   #weight_decay   #5e-5
        opt = SWA(optimizer)  
      # create learning schedule
        cosine_learning_schedule = create_cosine_learning_schedule(epochs, lr)
        run_train(model_name,mode,i)

170

--- [START IDENTIFIER] ----------------------------------------------------------------


	out_dir      = ./multimodal_Resnet_clinic_Normal_weight_file/0/



	<additional comments> ...  

	  - multimodal Resnet_clinic  

	  - simple augmentation 



** start Resnet_with_Uncertainty(clinic) training here! **

Round: 0, epoch: 0, Train Loss: 1.0521, Train Dia Acc: 0.2032, Train SPS Acc: 0.4259

Round: 0, epoch: 0, Valid Loss: 0.9529, Valid Dia Acc: 0.1935, Valid SPS Acc: 0.4941

Current Best Mean Acc is 0.45650407790047365
Round: 0, epoch: 1, Train Loss: 1.0637, Train Dia Acc: 0.2112, Train SPS Acc: 0.4197

Round: 0, epoch: 1, Valid Loss: 0.9484, Valid Dia Acc: 0.1984, Valid SPS Acc: 0.4976

Current Best Mean Acc is 0.4601626168118745
Round: 0, epoch: 2, Train Loss: 1.0410, Train Dia Acc: 0.2217, Train SPS Acc: 0.4371

Round: 0, epoch: 2, Valid Loss: 0.9419, Valid Dia Acc: 0.2341, Valid SPS Acc: 0.4906

Round: 0, epoch: 3, Train Loss: 1.0149, Train Dia Acc: 0.2570, Train SPS Acc: 0.

Round: 0, epoch: 41, Train Loss: 0.7524, Train Dia Acc: 0.6088, Train SPS Acc: 0.6514

Round: 0, epoch: 41, Valid Loss: 0.7889, Valid Dia Acc: 0.5187, Valid SPS Acc: 0.6300

Round: 0, epoch: 42, Train Loss: 0.7468, Train Dia Acc: 0.6104, Train SPS Acc: 0.6534

Round: 0, epoch: 42, Valid Loss: 0.7838, Valid Dia Acc: 0.5870, Valid SPS Acc: 0.6376

Round: 0, epoch: 43, Train Loss: 0.7517, Train Dia Acc: 0.6161, Train SPS Acc: 0.6415

Round: 0, epoch: 43, Valid Loss: 0.7780, Valid Dia Acc: 0.5740, Valid SPS Acc: 0.6444

Round: 0, epoch: 44, Train Loss: 0.7429, Train Dia Acc: 0.6329, Train SPS Acc: 0.6579

Round: 0, epoch: 44, Valid Loss: 0.7676, Valid Dia Acc: 0.6049, Valid SPS Acc: 0.6462

Round: 0, epoch: 45, Train Loss: 0.7409, Train Dia Acc: 0.6369, Train SPS Acc: 0.6569

Round: 0, epoch: 45, Valid Loss: 0.7625, Valid Dia Acc: 0.6016, Valid SPS Acc: 0.6551

Current Best Mean Acc is 0.6483740025177234
Round: 0, epoch: 46, Train Loss: 0.7329, Train Dia Acc: 0.6345, Train SPS Acc: 0.6527


Round: 0, epoch: 88, Train Loss: 0.6403, Train Dia Acc: 0.6771, Train SPS Acc: 0.7191

Round: 0, epoch: 88, Valid Loss: 0.7415, Valid Dia Acc: 0.5724, Valid SPS Acc: 0.6485

Round: 0, epoch: 89, Train Loss: 0.6498, Train Dia Acc: 0.7173, Train SPS Acc: 0.6912

Round: 0, epoch: 89, Valid Loss: 0.7580, Valid Dia Acc: 0.5967, Valid SPS Acc: 0.6441

Round: 0, epoch: 90, Train Loss: 0.6185, Train Dia Acc: 0.6900, Train SPS Acc: 0.7205

Round: 0, epoch: 90, Valid Loss: 0.7352, Valid Dia Acc: 0.5951, Valid SPS Acc: 0.6520

Round: 0, epoch: 91, Train Loss: 0.6378, Train Dia Acc: 0.6900, Train SPS Acc: 0.7135

Round: 0, epoch: 91, Valid Loss: 0.7355, Valid Dia Acc: 0.6000, Valid SPS Acc: 0.6588

Current Best Mean Acc is 0.6514227828238068
Round: 0, epoch: 92, Train Loss: 0.6271, Train Dia Acc: 0.6795, Train SPS Acc: 0.7223

Round: 0, epoch: 92, Valid Loss: 0.7436, Valid Dia Acc: 0.6016, Valid SPS Acc: 0.6562

Round: 0, epoch: 93, Train Loss: 0.6367, Train Dia Acc: 0.7108, Train SPS Acc: 0.7026


Round: 0, epoch: 131, Valid Loss: 0.7190, Valid Dia Acc: 0.5837, Valid SPS Acc: 0.6588

SWA Epoch: 131
Round: 0, epoch: 132, Train Loss: 0.5746, Train Dia Acc: 0.7237, Train SPS Acc: 0.7400

Round: 0, epoch: 132, Valid Loss: 0.7122, Valid Dia Acc: 0.6195, Valid SPS Acc: 0.6569

SWA Epoch: 132
Round: 0, epoch: 133, Train Loss: 0.5948, Train Dia Acc: 0.7092, Train SPS Acc: 0.7431

Round: 0, epoch: 133, Valid Loss: 0.7110, Valid Dia Acc: 0.6033, Valid SPS Acc: 0.6660

SWA Epoch: 133
Round: 0, epoch: 134, Train Loss: 0.5818, Train Dia Acc: 0.7253, Train SPS Acc: 0.7399

Round: 0, epoch: 134, Valid Loss: 0.7019, Valid Dia Acc: 0.6309, Valid SPS Acc: 0.6764

Current Best Mean Acc is 0.6707317257163729
SWA Epoch: 134
Round: 0, epoch: 135, Train Loss: 0.5834, Train Dia Acc: 0.7301, Train SPS Acc: 0.7367

Round: 0, epoch: 135, Valid Loss: 0.7065, Valid Dia Acc: 0.5935, Valid SPS Acc: 0.6620

SWA Epoch: 135
Round: 0, epoch: 136, Train Loss: 0.5751, Train Dia Acc: 0.7237, Train SPS Acc: 0.7414

R

Round: 0, epoch: 174, Valid Loss: 0.7080, Valid Dia Acc: 0.6163, Valid SPS Acc: 0.6620

SWA Epoch: 174
Round: 0, epoch: 175, Train Loss: 0.5604, Train Dia Acc: 0.7245, Train SPS Acc: 0.7527

Round: 0, epoch: 175, Valid Loss: 0.7143, Valid Dia Acc: 0.6114, Valid SPS Acc: 0.6576

SWA Epoch: 175
Round: 0, epoch: 176, Train Loss: 0.5309, Train Dia Acc: 0.7373, Train SPS Acc: 0.7658

Round: 0, epoch: 176, Valid Loss: 0.7173, Valid Dia Acc: 0.5967, Valid SPS Acc: 0.6597

SWA Epoch: 176
Round: 0, epoch: 177, Train Loss: 0.5419, Train Dia Acc: 0.7253, Train SPS Acc: 0.7657

Round: 0, epoch: 177, Valid Loss: 0.7178, Valid Dia Acc: 0.6114, Valid SPS Acc: 0.6606

SWA Epoch: 177
Round: 0, epoch: 178, Train Loss: 0.5480, Train Dia Acc: 0.7486, Train SPS Acc: 0.7546

Round: 0, epoch: 178, Valid Loss: 0.7114, Valid Dia Acc: 0.6293, Valid SPS Acc: 0.6648

SWA Epoch: 178
Round: 0, epoch: 179, Train Loss: 0.5516, Train Dia Acc: 0.7333, Train SPS Acc: 0.7627

Round: 0, epoch: 179, Valid Loss: 0.7072, Val

Round: 0, epoch: 217, Valid Loss: 0.7016, Valid Dia Acc: 0.6211, Valid SPS Acc: 0.6706

SWA Epoch: 217
Round: 0, epoch: 218, Train Loss: 0.5167, Train Dia Acc: 0.7398, Train SPS Acc: 0.7710

Round: 0, epoch: 218, Valid Loss: 0.7048, Valid Dia Acc: 0.6195, Valid SPS Acc: 0.6688

SWA Epoch: 218
Round: 0, epoch: 219, Train Loss: 0.5243, Train Dia Acc: 0.7502, Train SPS Acc: 0.7706

Round: 0, epoch: 219, Valid Loss: 0.7032, Valid Dia Acc: 0.6358, Valid SPS Acc: 0.6623

SWA Epoch: 219
Round: 0, epoch: 220, Train Loss: 0.5084, Train Dia Acc: 0.7470, Train SPS Acc: 0.7904

Round: 0, epoch: 220, Valid Loss: 0.7125, Valid Dia Acc: 0.6081, Valid SPS Acc: 0.6583

SWA Epoch: 220
Round: 0, epoch: 221, Train Loss: 0.5340, Train Dia Acc: 0.7373, Train SPS Acc: 0.7752

Round: 0, epoch: 221, Valid Loss: 0.7053, Valid Dia Acc: 0.6293, Valid SPS Acc: 0.6648

SWA Epoch: 221
Round: 0, epoch: 222, Train Loss: 0.5300, Train Dia Acc: 0.7205, Train SPS Acc: 0.7673

Round: 0, epoch: 222, Valid Loss: 0.7009, Val